In [1]:
pip install -U imbalanced-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.6/235.6 kB 2.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 6.6 MB/s eta 0:00:0000:01
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.0
    Uninstalling joblib-1.1.0:
      Successfully uninstalled joblib-1.1.0
  Attempting uninstall: imbalanced-learn
    Found existing installation: imbalanced-learn 0.9.1
    Uninstalling imbalanced-learn-0.9.1:
      Successfully uninstalled imbalanced-learn-0.9.1

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, confusion_matrix
import imblearn
from collections import Counter
from sklearn.metrics import roc_auc_score

In [3]:
df = pd.read_csv('cleaned_data_final.csv')
df.head(10)

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,nameOrigDest
0,1,0,9839.64,170136.00,160296.36,0.0,0.00,0,0
1,1,0,1864.28,21249.00,19384.72,0.0,0.00,0,0
2,1,1,181.00,181.00,0.00,0.0,0.00,1,1
3,1,2,181.00,181.00,0.00,21182.0,0.00,1,1
4,1,0,11668.14,41554.00,29885.86,0.0,0.00,0,0
5,1,0,7817.71,53860.00,46042.29,0.0,0.00,0,0
6,1,0,7107.77,183195.00,176087.23,0.0,0.00,0,0
7,1,0,7861.64,176087.23,168225.59,0.0,0.00,0,0
8,1,0,4024.36,2671.00,0.00,0.0,0.00,0,0
9,1,3,5337.77,41720.00,36382.23,41898.0,40348.79,0,1


In [4]:
x = df[['step', 'type', 'amount', 'oldbalanceOrg','newbalanceOrig', 'oldbalanceDest', 'newbalanceDest', 'nameOrigDest']]
y = df['isFraud']

In [5]:
# split train and test data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.33, random_state= 42)

# Original Dataset

In [6]:
gnb = GaussianNB()
gnb.fit(x_train, y_train)
y_pred = gnb.predict(x_test)

In [7]:
print(confusion_matrix(y_test, y_pred))

[[2083788   13201]
 [   2215     456]]


In [8]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00   2096989
           1       0.03      0.17      0.06      2671

    accuracy                           0.99   2099660
   macro avg       0.52      0.58      0.53   2099660
weighted avg       1.00      0.99      1.00   2099660



In [9]:
ROC_AUC = roc_auc_score(y_test, y_pred)

print('ROC AUC : {:.4f}'.format(ROC_AUC))

ROC AUC : 0.5822


# Undersampling case

In [12]:
from imblearn.under_sampling import RandomUnderSampler
rus=RandomUnderSampler(random_state=42)
x_rus, y_rus = rus.fit_resample(x_train, y_train)

print('original dataset shape:', Counter(y_train))
print('Resample dataset shape:', Counter(y_rus))

original dataset shape: Counter({0: 4257418, 1: 5526})
Resample dataset shape: Counter({0: 5526, 1: 5526})


In [13]:
gnb = GaussianNB()
gnb.fit(x_rus, y_rus)
y_pred_under = gnb.predict(x_test)

In [14]:
print(confusion_matrix(y_test, y_pred_under))

[[2014361   82628]
 [   1685     986]]


In [15]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00   2096989
           1       0.03      0.17      0.06      2671

    accuracy                           0.99   2099660
   macro avg       0.52      0.58      0.53   2099660
weighted avg       1.00      0.99      1.00   2099660



In [16]:
ROC_AUC = roc_auc_score(y_test, y_pred)

print('ROC AUC : {:.4f}'.format(ROC_AUC))

ROC AUC : 0.5822


# Oversampling case

In [17]:
from imblearn.over_sampling import RandomOverSampler
rus = RandomOverSampler(random_state=42)
x_rus_over, y_rus_over = rus.fit_resample(x_train, y_train)
print('original dataset shape:', Counter(y_train))
print('Resample dataset shape', Counter(y_rus_over))

original dataset shape: Counter({0: 4257418, 1: 5526})
Resample dataset shape Counter({0: 4257418, 1: 4257418})


In [18]:
gnb = GaussianNB()
gnb.fit(x_rus_over, y_rus_over)
y_pred_over = gnb.predict(x_test)

In [19]:
print(confusion_matrix(y_test, y_pred_over))

[[2029982   67007]
 [   1631    1040]]


In [20]:
print(classification_report(y_test, y_pred_over))

              precision    recall  f1-score   support

           0       1.00      0.97      0.98   2096989
           1       0.02      0.39      0.03      2671

    accuracy                           0.97   2099660
   macro avg       0.51      0.68      0.51   2099660
weighted avg       1.00      0.97      0.98   2099660



In [21]:
ROC_AUC = roc_auc_score(y_test, y_pred_over)

print('ROC AUC : {:.4f}'.format(ROC_AUC))

ROC AUC : 0.6787
